## Pyrlang_channel

In [1]:
from pyrlang_channel import Channel
import pyrlang_channel
from term import Atom
from asyncio import wait_for, TimeoutError

In [2]:
conn = pyrlang_channel.connect('associata@Beast', 'aas@Beast', 'aas_vis@Beast', 'aas_cookie')

In [3]:
ch = Channel(conn, 'ctrl')

In [4]:
await ch.send_backend_async((Atom('hello'), Atom('world'), 5, 4.3, "Joe?", {Atom('a'): Atom('b')}, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], {Atom('a'): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, True, False))

Send to unknown global_name_server ignored
rex unhandled info msg: (<1716216283.57.0 @ aas@Beast>, Atom('features_request'))


In [6]:
try:
    await wait_for(ch.receive_vis_async(), timeout=3)
except TimeoutError:
    print('Timeout')

Timeout


## Direct

In [1]:
from pyrlang import Node
from term import Atom
from pyrlang.process import Process

In [2]:
node = Node(node_name="py@Beast", cookie="aas_cookie")

In [3]:
class PassiveReceiver(Process):
    def __init__(self):
        super().__init__()
        self.get_node().register_name(self, Atom('ctrl'))

In [4]:
passive_receiver = PassiveReceiver()
pid = node.register_new_process(passive_receiver)

In [5]:
await node.send(sender=pid,
                    receiver=(Atom('aas@Beast'), Atom('ctrl')),
                    message=pid)

rex unhandled info msg: (<1717410274.57.0 @ aas@Beast>, Atom('features_request'))
Send to unknown global_name_server ignored


<1717413437.0.2 @ py@Beast>: Unhandled msg hellojoe


In [9]:
await passive_receiver.inbox_.get()

<1717413437.0.2 @ py@Beast>: Unhandled msg hellojoe
<1717413437.0.2 @ py@Beast>: Unhandled msg hellojoe


CancelledError: 

In [7]:
await passive_receiver.receive()

<1717413437.0.2 @ py@Beast>: Unhandled msg hellojoe
<1717413437.0.2 @ py@Beast>: Unhandled msg hellojoe


CancelledError: 

In [3]:
import numpy as np

f = np.array([0.1])[0]

In [5]:
fake_pid = node.register_new_process()
await node.send(sender=fake_pid,
                    receiver=(Atom('associata@Beast'), Atom('ctrl')),
                    message=float(f))

In [3]:
async def example_main(node):
    fake_pid = node.register_new_process()

    # To be able to send to Erlang shell by name first give it a registered
    # name: `erlang:register(shell, self()).`
    # To see an incoming message in shell: `flush().`
    await node.send(sender=fake_pid,
                    receiver=(Atom('associata@Beast'), Atom('ctrl')),
                    message=Atom('hellofromjoepyter'))
    print('Message sent to Erlang shell')

In [4]:
ev = node.get_loop()
ev.create_task(example_main(node))
# node.run()

<Task pending name='Task-13' coro=<example_main() running at C:\Users\adams\AppData\Local\Temp\ipykernel_9440\1347485222.py:1>>

rex unhandled info msg: (<1716203536.57.0 @ associata@Beast>, Atom('features_request'))
Send to unknown global_name_server ignored


Message sent to Erlang shell


In [6]:
ev = node.get_loop()
await ev.create_task(example_main(node))

Message sent to Erlang shell


In [8]:
from pyrlang.process import Process

class Receiver(Process):
    def __init__(self):
        super().__init__()
        self.get_node().register_name(self, Atom('associata'))
        print('Receiver created')
    
    def handle_one_inbox_message(self, msg):
        print('Received message', msg)

In [9]:
proc = Receiver()

Receiver created


In [10]:
await node.send(sender=proc.pid_,
                    receiver=(Atom('aasociata@Beast'), Atom('ctrl')),
                    message=f)

In [6]:
fake_pid = node.register_new_process()
await node.send(sender=fake_pid,
                    receiver=(Atom('py@Beast'), Atom('associata')),
                    message=Atom('localmsg'))

Received message localmsg


In [7]:
fake_pid = node.register_new_process()
await node.send(sender=fake_pid,
                    receiver=(Atom('associata@Beast'), Atom('ctrl')),
                    message=Atom('joe'))

In [12]:
import asyncio
asyncio.run(example_main(node))

RuntimeError: asyncio.run() cannot be called from a running event loop

## Associata

In [1]:
import associata

In [2]:
await associata.init()

Send to unknown global_name_server ignored
rex unhandled info msg: (<1717410272.57.0 @ aas@Beast>, Atom('features_request'))


In [3]:
agds = await associata.create_agds()

In [4]:
await agds.add_numerical_vng('plato', 0.5)
await agds.add_numerical_vng('ibu', 0.5)
await agds.add_numerical_vng('ebc', 0.5)

In [5]:
await agds.add_observation({'plato': 12, 'ibu': 25, 'ebc': 30})
await agds.add_observation({'plato': 14, 'ibu': 35, 'ebc': 50})
await agds.add_observation({'plato': 12, 'ibu': 45, 'ebc': 50})

Adding observation: (Atom('add_observation'), {'plato': 12, 'ibu': 25, 'ebc': 30})
Adding observation: (Atom('add_observation'), {'plato': 14, 'ibu': 35, 'ebc': 50})
Adding observation: (Atom('add_observation'), {'plato': 12, 'ibu': 45, 'ebc': 50})


In [6]:
await agds.reset_excitation()

In [6]:
inf = associata.InferenceSetup()
inf.stimulate_vn('ibu', 12)
inf.stimulate_vn('ebc', 40)

In [7]:
await agds.infere(inf, 2)

In [8]:
await agds.get_excitations_for_vng('ibu')

NodeException: Don't know how to send to None
Task exception was never retrieved
future: <Task finished name='Task-39' coro=<BaseDistProtocol.on_passthrough_message() done, defined at c:\users\adams\lib\pyrlang\pyrlang\pyrlang\dist_proto\base_dist_protocol.py:224> exception=NodeException("Don't know how to send to None")>
Traceback (most recent call last):
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\dist_proto\base_dist_protocol.py", line 240, in on_passthrough_message
    return await n.send(sender=control_term[1],
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\node.py", line 253, in send
    raise NodeException("Don't know how to send to %s" % receiver)
pyrlang.errors.NodeException: Don't know how to send to None


In [10]:
await agds.get_excitations_for_ong()

NodeException: Don't know how to send to None
Task exception was never retrieved
future: <Task finished name='Task-40' coro=<BaseDistProtocol.on_passthrough_message() done, defined at c:\users\adams\lib\pyrlang\pyrlang\pyrlang\dist_proto\base_dist_protocol.py:224> exception=NodeException("Don't know how to send to None")>
Traceback (most recent call last):
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\dist_proto\base_dist_protocol.py", line 240, in on_passthrough_message
    return await n.send(sender=control_term[1],
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\node.py", line 253, in send
    raise NodeException("Don't know how to send to %s" % receiver)
pyrlang.errors.NodeException: Don't know how to send to None


In [7]:
await agds.stop()

Task exception was never retrieved
future: <Task finished name='Task-28' coro=<Process.handle_signals() done, defined at c:\users\adams\lib\pyrlang\pyrlang\pyrlang\process.py:178> exception=KeyError(<1717410268.0.3 @ associata@Beast>)>
Traceback (most recent call last):
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\process.py", line 184, in handle_signals
    self._on_exit_signal(reason)
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\process.py", line 248, in _on_exit_signal
    n.on_exit_process(self.pid_, reason)
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\node.py", line 146, in on_exit_process
    del self.processes_[exiting_pid]
        ~~~~~~~~~~~~~~~^^^^^^^^^^^^^
KeyError: <1717410268.0.3 @ associata@Beast>


In [3]:
await associata.stop()

Error: Transport is None in _send_packet4 [internal error] (state conn)
DistributionError: Transport is None in _send_packet4 [internal error]
Task exception was never retrieved
future: <Task finished name='Task-22' coro=<BaseDistProtocol.listen_on_inbox() done, defined at c:\users\adams\lib\pyrlang\pyrlang\pyrlang\dist_proto\base_dist_protocol.py:300> exception=DistributionError('Transport is None in _send_packet4 [internal error]')>
Traceback (most recent call last):
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\dist_proto\base_dist_protocol.py", line 303, in listen_on_inbox
    self._handle_one_inbox_message(msg)
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\dist_proto\base_dist_protocol.py", line 338, in _handle_one_inbox_message
    return self._control_message(ctrl, msg)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\users\adams\lib\pyrlang\pyrlang\pyrlang\dist_proto\base_dist_protocol.py", line 409, in _control_message
    self._send_packet4(packet)
  File "c:\